# Function for Loading the Data from a REST-API for a sp

In [127]:
# Imports:
import requests
from datetime import datetime, timedelta
import pandas as pd

In [128]:
def load_data(lat_start, lat_end, long_start, long_end, start_year, start_month, start_day, delta_hours, combined = True):
    '''Function for loading the data out of the REST-API'''
    
    '''INPUT:'''
    
    '''lat_start:                          latitude range starting point, type: byte'''
    '''lat_end:                            latitude range ending point, type: byte'''
    '''long_start:                         longitude range starting point, type: byte'''
    '''long_end:                           longitude range ending point, type: byte'''
    '''start_year, start_month, start_day: year / month / day of the measurement to start, type: byte'''
    '''delta_hours:                        time delta to calculate time space of measurement, type: byte'''
    '''combined:                           if True, append both (P1 and P1) in one large data frame, type: boolean, default: True'''
    
    '''OUPUT:'''
    
    '''Depending on 'combined', either both (P1 and P2) dataframes are outputted seperately or combined'''
    
    '''Import Data from REST_API'''
    # Basic parameters
    base_url='http://sensordata.gwdg.de/api/' # Select Endpoint (P1 or P2)
    endpoint_url_P1='measurements/P1'
    endpoint_url_P2='measurements/P2'

    # Select geo-coordinates (values are just examples ..)
    latrange=[lat_start, lat_end]
    longrange=[long_start, long_end]

    # Select time range (values are just examples ..)
    start_date = datetime(start_year, start_month, start_day)
    end_date = (start_date + timedelta(hours = delta_hours))

    # Build the query
    mydata = '{"timeStart": "'+start_date.strftime("%Y-%m-%dT%H:%M:%SZ")+'",' + \
             '"timeEnd": "'+end_date.strftime("%Y-%m-%dT%H:%M:%SZ")+'", "area":  \
             {"coordinates":['+str(latrange)+','+str(longrange)+']}}'

    # Run the query
    response_P1 = requests.post(base_url + endpoint_url_P1, data=mydata)
    response_P2 = requests.post(base_url + endpoint_url_P2, data=mydata)
    
    '''Initialize data frames'''
    j_P1 = response_P1.json()                                                                            # convert REST-API data to json at first
    del j_P1[1]                                                                                          # delete 'sensor' string, that causes errors
    df_P1 = pd.DataFrame(j_P1[1], columns =j_P1[0])                                                      # put all in pandas data frame
    df_P1 = df_P1.rename(columns={"P1": "measurement"})                                                  # Change column name for better overview
    df_P1['Sensor'] = "P1"                                                                               # Add column with sensor description
    df_P1['PM'] = "PM10"                                                                                 # Add column with official sensor description
    df_P1 = df_P1.reindex(columns = ["measurement", "time", "lat", "lon", "Sensor", "PM", "sensor_id"])  # rearranging column names for better overview

    j_P2 = response_P2.json()
    del j_P2[1]
    df_P2 = pd.DataFrame(j_P2[1], columns =j_P2[0])
    df_P2 = df_P2.rename(columns={"P2": "measurement"})
    df_P2['Sensor'] = "P2"
    df_P2['PM'] = "PM2.5"
    df_P2 = df_P2.reindex(columns = ["measurement", "time", "lat", "lon", "Sensor", "PM", "sensor_id"])
    
    if combined == False:
        return df_P1, df_P2
    else:
        df_total = df_P1.append(df_P2, sort = False)
        return df_total

In [129]:
# Example
df = load_data(lat_start = 51, lat_end = 52, long_start = 9, long_end = 10, start_year = 2018, start_month = 4, start_day = 30, delta_hours = 1, combined = True)
df

,measurement,time,lat,lon,Sensor,PM,sensor_id
0,18.43,2018-04-30T00:00:03Z,51.732,9.032,P1,PM10,9946
1,9.17,2018-04-30T00:00:11Z,51.340,9.429,P1,PM10,4784
2,NaN,2018-04-30T00:00:12Z,51.340,9.429,P1,PM10,4785
3,12.87,2018-04-30T00:00:14Z,51.520,9.954,P1,PM10,11998
4,NaN,2018-04-30T00:00:14Z,51.520,9.954,P1,PM10,11999
...,...,...,...,...,...,...,...
1082,9.00,2018-04-30T00:59:50Z,51.290,9.641,P2,PM2.5,4973
1083,NaN,2018-04-30T00:59:51Z,51.290,9.641,P2,PM2.5,4974
1084,11.17,2018-04-30T00:59:54Z,51.854,9.672,P2,PM2.5,9870
1085,12.00,2018-04-30T00:59:57Z,51.561,9.987,P2,PM2.5,12334
